In [1]:
!pip install -q transformers torch pillow accelerate
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -albumentations
!pip install opencv-python-headless

import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define transform for training
train_transform = A.Compose([
    A.Resize(384, 384),
    A.RandomBrightnessContrast(p=0.3),
    A.GaussianBlur(p=0.2),
    A.Rotate(limit=5, p=0.4),
    A.Normalize(),  # important: normalize image
    ToTensorV2(),   # convert to PyTorch tensor
])

# Define transform for eval (no augmentation, just resize/normalize)
eval_transform = A.Compose([
    A.Resize(384, 384),
    A.Normalize(),
    ToTensorV2(),
])


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -a


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
!pip install -U transformers
import os
import cv2
import torch
from PIL import Image
from transformers import (
    TrOCRProcessor,
    VisionEncoderDecoderModel,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    TrainerCallback,
)
from torch.utils.data import Dataset
from google.colab import drive

# Define Dataset class
class PlateDataset(Dataset):
    def __init__(self, root_dir, processor, max_target_length=24, transform=None):
        self.root_dir = root_dir
        self.image_paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith((".jpg", ".png"))]
        self.processor = processor
        self.max_target_length = max_target_length
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)

        label = os.path.basename(image_path).split(".")[0].strip()
        label = label.split("(")[0].strip()
        label += self.processor.tokenizer.eos_token

        if len(label) < self.max_target_length:
            label += " " * (self.max_target_length - len(label))

        encoding = self.processor(
            images=image,
            text=label,
            return_tensors="pt",
            padding="longest",
            max_length=self.max_target_length,
            truncation=True,
        )

        labels = encoding["labels"].squeeze()
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        return {
            "pixel_values": encoding["pixel_values"].squeeze(),
            "labels": labels,
        }


# Define PredictionCallback class
class PredictionCallback(TrainerCallback):
    def __init__(self, processor, eval_dataset, num_samples=3):
        self.processor = processor
        self.eval_dataset = eval_dataset
        self.num_samples = num_samples

    def on_evaluate(self, args, state, control, **kwargs):
        model = kwargs['model']
        indices = torch.randperm(len(self.eval_dataset))[:self.num_samples]

        print("\nSample Prediction:")
        for idx in indices:
            sample = self.eval_dataset[idx]
            pixel_values = sample["pixel_values"].unsqueeze(0).to(model.device)

            generated_ids = model.generate(
                pixel_values,
                max_length=32,
                num_beams=5,
                early_stopping=False,
                length_penalty=1.5,
            )

            pred_text = self.processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

            label_ids = sample["labels"]
            label_ids = label_ids[label_ids != -100]
            true_text = self.processor.tokenizer.decode(label_ids, skip_special_tokens=True)

            print(f"Sample {idx}: Predict='{pred_text}' | Actual='{true_text}'")


def train():
    drive.mount('/content/drive')

    processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
    model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")
    model.config.decoder_start_token_id = processor.tokenizer.bos_token_id
    model.config.pad_token_id = processor.tokenizer.pad_token_id

    train_dataset = PlateDataset("/content/drive/MyDrive/Latest/OCR_Dataset/train", processor, max_target_length=32)
    eval_dataset = PlateDataset("/content/drive/MyDrive/Latest/OCR_Dataset/val", processor, max_target_length=32)

    training_args = Seq2SeqTrainingArguments(
        output_dir="./trocr_plate_output",
        per_device_train_batch_size=12,
        num_train_epochs=30,
        fp16=True,
        eval_strategy="steps",
        eval_steps=500,
        logging_steps=100,
        learning_rate=3e-5,
        warmup_steps=300,
        save_steps=500,
        report_to="none",
        load_best_model_at_end=True,
        save_total_limit=2,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        weight_decay=0.01,
        gradient_accumulation_steps=2,
    )

    pred_callback = PredictionCallback(processor, eval_dataset, num_samples=3)

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=lambda x: {
            "pixel_values": torch.stack([item["pixel_values"] for item in x]),
            "labels": torch.nn.utils.rnn.pad_sequence(
                [item["labels"] for item in x],
                batch_first=True,
                padding_value=processor.tokenizer.pad_token_id,
            )
        },
        callbacks=[pred_callback],
    )

    trainer.train()

    trainer.save_model("/content/drive/MyDrive/trocr_plate_model_complete")
    processor.save_pretrained("/content/drive/MyDrive/trocr_plate_model_complete")
    print("Model saved to Google Drive!")


if __name__ == "__main__":
    # First train the model (or comment this if already trained)
    train()

    # Then run inference on test data:
    model_path = "/content/drive/MyDrive/trocr_plate_model_complete"
    test_folder_path = "/content/drive/MyDrive/Latest/OCR_Dataset/test"
    test_inference_folder(model_path, test_folder_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.